In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.constraints import *
from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.table.grammar import *
from spatial_scene_grammars_examples.table.grammar_decoration import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

from pydrake.all import Quaternion, RigidTransform

import meshcat
import meshcat.geometry as meshcat_geom

In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Meshcat url:  http://127.0.0.1:7001/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [18]:
trees_to_draw = []
#with open("target_dataset_examples.pickle", "rb") as f:
#with open("structure_constraint_examples.pickle", "rb") as f:
#with open("pre_fit_grammar_draws.pickle", "rb") as f:
with open("post_fit_grammar_draws.pickle", "rb") as f:
#with open("baseline_post_fit_grammar_draws.pickle", "rb") as f:
    while 1:
        try:
            #trees_to_draw.append(pickle.load(f))
            trees_to_draw.append(pickle.load(f)[0])
        except EOFError:
            break
print("Loaded %d environments" % len(trees_to_draw))
trees_to_draw = trees_to_draw[:30]

Loaded 50 environments


In [19]:
tree = trees_to_draw[0]

# Using the set of decoration mapping rules, create a new set of decoration
# nodes and recursively expand them by following their production rules. Add
# these to scene tree and project the whole thing to feasibility with forward
# sim.
decorated_tree = deepcopy(tree)
decorated_tree = apply_decoration_rules_to_tree(decorated_tree, decoration_mapping)

In [20]:
#draw_scene_tree_contents_meshcat(decorated_tree, zmq_url=vis.window.zmq_url,  prefix="tree/contents")
#draw_scene_tree_structure_meshcat(decorated_tree, zmq_url=vis.window.zmq_url,  prefix="tree/structure",
#                                  with_triad=True, linewidth=30, node_sphere_size=0.02)

In [21]:
# Simulate forwards; everything but the new stuff should already be frozen.
projected_tree = project_tree_to_feasibility(decorated_tree, do_forward_sim=True)

In [22]:
draw_scene_tree_contents_meshcat(projected_tree, zmq_url=vis.window.zmq_url,  prefix="projected_tree/contents")
draw_scene_tree_structure_meshcat(projected_tree, zmq_url=vis.window.zmq_url,  prefix="projected_tree/structure",
                                  with_triad=True, linewidth=30, node_sphere_size=0.02)

# Apply that pipeline to generate a dataset of decorated scenes.

In [23]:
# This one saves out the post-fit examples, before and after the physics projection step.
dataset_save_file = "post_fit_grammar_draws_decorated.pickle"
for tree in tqdm(trees_to_draw, desc="Samples"):
    decorated_tree = deepcopy(tree)
    decorated_tree = apply_decoration_rules_to_tree(decorated_tree, decoration_mapping)
    projected_tree = project_tree_to_feasibility(decorated_tree, do_forward_sim=True)
    
    with open(dataset_save_file, "a+b") as f:
        pickle.dump((projected_tree, _), f)

Samples:   0%|          | 0/30 [00:00<?, ?it/s]

Logfile: 

         S N O P T  7.4-1.2  (Feb 2015)
1
 
 SNMEMA EXIT 100 -- finished successfully
 SNMEMA INFO 104 -- memory requirements estimated
1
 
 Parameters

 Files
 -----
 Solution file..........         0       Old basis file ........         0       Standard input.........         5
 Insert file............         0       New basis file ........         0       (Printer)..............        10
 Punch file.............         0       Backup basis file......         0       (Specs file)...........         0
 Load file..............         0       Dump file..............         0       Standard output........         6

 Frequencies
 -----------
 Print frequency........       100       Check frequency........        60       Save new basis map.....       100
 Summary frequency......       100       Factorization frequency        50       Expand frequency.......     10000

 QP subproblems
 --------------
 QPsolver Cholesky......
 Scale tolerance........     0.900       Minor 